# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 8 2022 12:34PM,243675,12,HH-36314,Hush Hush,Released
1,Aug 8 2022 12:34PM,243675,12,HH-36315,Hush Hush,Released
2,Aug 8 2022 12:31PM,243674,10,MTR-22-0243,"Senseonics, Incorporated",Released
3,Aug 8 2022 12:24PM,243673,10,8376863,"ClearSpec, LLC",Released
4,Aug 8 2022 12:17PM,243671,10,MSP212903,"Methapharm, Inc.",Released
5,Aug 8 2022 12:17PM,243671,10,MSP212923,"Methapharm, Inc.",Released
6,Aug 8 2022 12:17PM,243671,10,MSP212939,"Methapharm, Inc.",Released
7,Aug 8 2022 12:17PM,243671,10,MSP212942,"Methapharm, Inc.",Released
8,Aug 8 2022 12:16PM,243669,10,MSP212928,"Methapharm, Inc.",Released
9,Aug 8 2022 12:16PM,243669,10,MSP212929,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,243669,Released,18
29,243671,Released,4
30,243673,Released,1
31,243674,Released,1
32,243675,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243669,NaN,18.0
243671,NaN,4.0
243673,NaN,1.0
243674,NaN,1.0
243675,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243608,0.0,2.0
243612,1.0,0.0
243617,0.0,1.0
243623,13.0,41.0
243627,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243608,0,2
243612,1,0
243617,0,1
243623,13,41
243627,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243608,0,2
1,243612,1,0
2,243617,0,1
3,243623,13,41
4,243627,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243608,,2
1,243612,1,
2,243617,,1
3,243623,13,41
4,243627,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 8 2022 12:34PM,243675,12,Hush Hush
2,Aug 8 2022 12:31PM,243674,10,"Senseonics, Incorporated"
3,Aug 8 2022 12:24PM,243673,10,"ClearSpec, LLC"
4,Aug 8 2022 12:17PM,243671,10,"Methapharm, Inc."
8,Aug 8 2022 12:16PM,243669,10,"Methapharm, Inc."
26,Aug 8 2022 11:59AM,243667,10,"Citieffe, Inc."
27,Aug 8 2022 11:56AM,243666,10,"Citieffe, Inc."
28,Aug 8 2022 11:51AM,243665,10,Ivaoes Animal Health
29,Aug 8 2022 11:51AM,243664,10,"Nextsource Biotechnology, LLC"
30,Aug 8 2022 11:23AM,243662,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 8 2022 12:34PM,243675,12,Hush Hush,,2
1,Aug 8 2022 12:31PM,243674,10,"Senseonics, Incorporated",,1
2,Aug 8 2022 12:24PM,243673,10,"ClearSpec, LLC",,1
3,Aug 8 2022 12:17PM,243671,10,"Methapharm, Inc.",,4
4,Aug 8 2022 12:16PM,243669,10,"Methapharm, Inc.",,18
5,Aug 8 2022 11:59AM,243667,10,"Citieffe, Inc.",,1
6,Aug 8 2022 11:56AM,243666,10,"Citieffe, Inc.",,1
7,Aug 8 2022 11:51AM,243665,10,Ivaoes Animal Health,,1
8,Aug 8 2022 11:51AM,243664,10,"Nextsource Biotechnology, LLC",,1
9,Aug 8 2022 11:23AM,243662,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 12:34PM,243675,12,Hush Hush,2,
1,Aug 8 2022 12:31PM,243674,10,"Senseonics, Incorporated",1,
2,Aug 8 2022 12:24PM,243673,10,"ClearSpec, LLC",1,
3,Aug 8 2022 12:17PM,243671,10,"Methapharm, Inc.",4,
4,Aug 8 2022 12:16PM,243669,10,"Methapharm, Inc.",18,
5,Aug 8 2022 11:59AM,243667,10,"Citieffe, Inc.",1,
6,Aug 8 2022 11:56AM,243666,10,"Citieffe, Inc.",1,
7,Aug 8 2022 11:51AM,243665,10,Ivaoes Animal Health,1,
8,Aug 8 2022 11:51AM,243664,10,"Nextsource Biotechnology, LLC",1,
9,Aug 8 2022 11:23AM,243662,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 12:34PM,243675,12,Hush Hush,2,
1,Aug 8 2022 12:31PM,243674,10,"Senseonics, Incorporated",1,
2,Aug 8 2022 12:24PM,243673,10,"ClearSpec, LLC",1,
3,Aug 8 2022 12:17PM,243671,10,"Methapharm, Inc.",4,
4,Aug 8 2022 12:16PM,243669,10,"Methapharm, Inc.",18,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 12:34PM,243675,12,Hush Hush,2.0,NaN
1,Aug 8 2022 12:31PM,243674,10,"Senseonics, Incorporated",1.0,NaN
2,Aug 8 2022 12:24PM,243673,10,"ClearSpec, LLC",1.0,NaN
3,Aug 8 2022 12:17PM,243671,10,"Methapharm, Inc.",4.0,NaN
4,Aug 8 2022 12:16PM,243669,10,"Methapharm, Inc.",18.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 12:34PM,243675,12,Hush Hush,2.0,0.0
1,Aug 8 2022 12:31PM,243674,10,"Senseonics, Incorporated",1.0,0.0
2,Aug 8 2022 12:24PM,243673,10,"ClearSpec, LLC",1.0,0.0
3,Aug 8 2022 12:17PM,243671,10,"Methapharm, Inc.",4.0,0.0
4,Aug 8 2022 12:16PM,243669,10,"Methapharm, Inc.",18.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3654790,103.0,27.0
12,243675,2.0,0.0
15,730944,23.0,12.0
16,243617,1.0,0.0
19,730909,10.0,16.0
21,974638,3.0,1.0
22,243627,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3654790,103.0,27.0
1,12,243675,2.0,0.0
2,15,730944,23.0,12.0
3,16,243617,1.0,0.0
4,19,730909,10.0,16.0
5,21,974638,3.0,1.0
6,22,243627,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,103.0,27.0
1,12,2.0,0.0
2,15,23.0,12.0
3,16,1.0,0.0
4,19,10.0,16.0
5,21,3.0,1.0
6,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,103.0
1,12,Released,2.0
2,15,Released,23.0
3,16,Released,1.0
4,19,Released,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Executing,27.0,0.0,12.0,0.0,16.0,1.0,0.0
Released,103.0,2.0,23.0,1.0,10.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Executing,27.0,0.0,12.0,0.0,16.0,1.0,0.0
1,Released,103.0,2.0,23.0,1.0,10.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Executing,27.0,0.0,12.0,0.0,16.0,1.0,0.0
1,Released,103.0,2.0,23.0,1.0,10.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()